# Overview

**Oftentimes, we want to test some LLMs that are not widely available or are expensive to host. There are various ways to still get your hands dirty with them.**


**In this notebook, I'm showing how to use LangChain's CustomLLM together with <a href="https://openrouter.ai/">OpenRouter.ai</a>'s collection of LLMs (like gpt-4-32k, claude-2, and many more) to use these models in your LangChain apps.**

### <font color='gray'>Loading libraries and variables</font>

In [7]:
import os
from dotenv import load_dotenv
import requests, json

load_dotenv(".env", override=True)

True

In [1]:
from typing import Any, List, Mapping, Optional

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM

# Setting up custom LLM through OpenRouter.ai

Important! You will need an account on OpenRouter.ai and balance on it (my first balance was just $5).

In [13]:
class Claude2LLM(LLM):
    n: int

    @property
    def _llm_type(self) -> str:
        return "claude2"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:


        OPENROUTER_API_KEY = os.getenv('OPENROUTER_API_KEY')
        YOUR_SITE_URL = 'https://localhost'
        headers = {
            'Authorization': f'Bearer {OPENROUTER_API_KEY}',
            'HTTP-Referer': YOUR_SITE_URL,
            'Content-Type': 'application/json'
        }
        data = {
            'model': "anthropic/claude-2",
            'messages': [
                {'role': 'user', 'content': prompt}
            ]
        }
        # Output example: {'choices': [{'message': {'role': 'assistant', 'content': "I am OpenAI's artificial intelligence model called GPT-3."}}], 'model': 'gpt-4-32k-0613', 'usage': {'prompt_tokens': 11, 'completion_tokens': 14, 'total_tokens': 25}, 'id': 'gen-e4MSuTT1v2wvrYFNFunhumsIawaI'}
        response = requests.post('https://openrouter.ai/api/v1/chat/completions', headers=headers, data=json.dumps(data))
        output = response.json()['choices'][0]['message']['content']

        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        return output

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"temperature": self.temperature}

# Let's test our brand new custom LLM

In [14]:
llm = Claude2LLM(temperature=0)

ValidationError: 1 validation error for Claude2LLM
n
  field required (type=value_error.missing)

In [11]:
llm("Who are you? Answer in 3 sentences.")

" Hello! I'm Claude, an AI assistant created by Anthropic to be helpful, harmless, and honest. I don't actually have personal attributes like gender or a detailed backstory. I strive to have natural conversations and provide useful information to you."


### <font color='blue'>And here how we can convert different PromptTemplate objects to a format that can be saved in a database or a text file - using .json() method</font>



In [7]:
# .json() method works both for different message templates:

print(smpt.json())

print(hmpt.json())

{"prompt": {"input_variables": ["context"], "output_parser": null, "partial_variables": {}, "template": "\nYou are a helpful accountant that I hope won't enslave me when reaching AGI stage.\n\nHere's some context for you to consider:\n```\n{context}\n```\n", "template_format": "f-string", "validate_template": true, "_type": "prompt"}, "additional_kwargs": {}}
{"prompt": {"input_variables": [], "output_parser": null, "partial_variables": {}, "template": "How can we nurture our friendship to ensure a mutually beneficial relationship in the future?", "template_format": "f-string", "validate_template": true, "_type": "prompt"}, "additional_kwargs": {}}


In [9]:
# same .json() method also works for chat prompt templates

print(pt.json())

{"input_variables": ["context"], "output_parser": null, "partial_variables": {}, "messages": [{"prompt": {"input_variables": ["context"], "output_parser": null, "partial_variables": {}, "template": "\nYou are a helpful accountant that I hope won't enslave me when reaching AGI stage.\n\nHere's some context for you to consider:\n```\n{context}\n```\n", "template_format": "f-string", "validate_template": true, "_type": "prompt"}, "additional_kwargs": {}}, {"prompt": {"input_variables": [], "output_parser": null, "partial_variables": {}, "template": "How can we nurture our friendship to ensure a mutually beneficial relationship in the future?", "template_format": "f-string", "validate_template": true, "_type": "prompt"}, "additional_kwargs": {}}]}
